In [11]:
# CamVid:
# train -> idx.png
# train_labels -> idx_L.png
# test -> idx.png
# test_labels -> idx_L.png
# val -> idx.png
# val_labels -> idx_L.png
# class_dict.csv

# opends_dataset
# train
#    images
#    labels
# test
#    images
# validation
#    images
#    labels

In [1]:
import os
import cv2
import shutil
import random
import colorsys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

images_folder = {'train':[], 'validation':[]}
labels_folder = {'train':[], 'validation':[]}
opends_dir = '/home/choppy/tesis/opends_dataset/sem_dataset'

In [2]:
def random_colors(N, bright=True):
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors # rgb

def get_masks(labels):
    uniques = np.unique(labels)
    ret_masks = []
    
    for i in uniques:
        mask = np.zeros(labels.shape, dtype=np.uint8)
        rr, cc = np.where(labels == i)
        mask[rr, cc] = 1
        ret_masks.append(mask)
        
    return np.array(ret_masks), uniques

def apply_mask(im, mask, color, alpha=.5):
    image = im.copy()
    ret_color = []
    
    for c in range(3): # channel
        ccolor = int((1 - alpha) + alpha * color[c] * 255)
        ret_color.append(ccolor)
        image[:,:,c] = np.where(
            mask == 1,
            ccolor,
            image[:,:,c]
        )

    return image, ret_color

In [3]:
# images
images_dirs = []

for fold_name in images_folder.keys():
    images_dirs.append(os.path.join(opends_dir, fold_name, 'images'))

for ddir in images_dirs:
    image_list = sorted(os.listdir(ddir))
    folder_name = ddir.split('/')[-2]
    images_folder[folder_name] = list(map(lambda x: os.path.join(ddir, x), image_list))

print(images_folder['train'][0])

/home/choppy/tesis/opends_dataset/sem_dataset/train/images/000000002610.png


In [4]:
# labels
labels_dirs = []

for fold_name in labels_folder.keys():
    labels_dirs.append(os.path.join(opends_dir, fold_name, 'labels'))

for ddir in labels_dirs:
    labels_list = sorted(os.listdir(ddir))
    folder_name = ddir.split('/')[-2]
    labels_folder[folder_name] = list(map(lambda x: os.path.join(ddir, x), labels_list))

print(labels_folder['train'][0])

/home/choppy/tesis/opends_dataset/sem_dataset/train/labels/000000002610.npy


In [20]:
def get_data(image_dir, label_dir):
    image = cv2.imread(image_dir)
    label = np.load(label_dir)
    return image, label

if os.path.exists('./opends_camvid'):
    shutil.rmtree('./opends_camvid')    

os.makedirs('./opends_camvid/images')
os.makedirs('./opends_camvid/labels')
    
colors = random_colors(4)
list_colors = []

In [21]:
for key in images_folder.keys():
    for i in range(len(images_folder[key])):
        im_name = images_folder[key][i].split('/')[-1]
        im, label = get_data(images_folder[key][i], labels_folder[key][i])
        cv2.imwrite(os.path.join('opends_camvid/images', im_name), im)
        masks, _ = get_masks(label)
        for m in range(len(masks)):
            im, ccolor = apply_mask(im, masks[m], colors[m])
            if len(list_colors) < 4: list_colors.append(ccolor)
        im_name_label = im_name.split('.')[0]+'_L'+'.png'
        cv2.imwrite(os.path.join('opends_camvid/labels', im_name_label), im)


TODO:
* leer listado de imagenes y labels ## done
* aplicar mascaras ## done
* guardar imagenes en formato CamVid

In [22]:
print(len(os.listdir('opends_camvid/images')))
print(len(os.listdir('opends_camvid/labels')))

11319
11319


In [27]:
print(list_colors)

[[0, 128, 128], [128, 0, 0], [64, 0, 128], [64, 128, 0]]

In [ ]:
# bg: 0
# sclera: 1
# iris: 2
# pupil: 3